# argument order

In [30]:
import pandas as pd
from cosmograph import cosmo
from i2 import Sig
from i2.signatures import parameter_to_dict

cosmo_sig = Sig(cosmo)
pd.DataFrame(map(parameter_to_dict, cosmo_sig.params))


,name,kind,default,annotation
0,data,1,None,<class 'inspect._empty'>
1,points,3,None,<class 'object'>
2,links,3,None,<class 'object'>
3,disable_simulation,3,None,<class 'bool'>
4,simulation_decay,3,None,<class 'float'>
...,...,...,...,...
91,clicked_point_id,3,None,<class 'str'>
92,selected_point_indices,3,None,list[int]
93,selected_point_ids,3,None,list[str]
94,changePoints,3,None,"typing.Callable[[typing.Dict[str, typing.Any]]..."


In [81]:
from cosmograph._code_sync import order_cosmo_params, print_param_spec_code_lines

old_sig = Sig(cosmo)
new_sig = order_cosmo_params(cosmo)
print_param_spec_code_lines(new_sig)


	data=None,
	*,
	points : object = None,
	links : object = None,
	point_x_by : str = None,
	point_y_by : str = None,
	point_size_by : str = None,
	point_color_by : str = None,
	point_label_by : str = None,
	point_color : Union[str, list[float]] = None,
	point_greyout_opacity : float = None,
	point_size : float = None,
	point_size_scale : float = None,
	point_sampling_distance : int = None,
	point_id_by : str = None,
	point_index_by : str = None,
	point_size_range : list[float] = None,
	point_label_weight_by : str = None,
	point_cluster_by : str = None,
	point_cluster_strength_by : str = None,
	point_include_columns : list[str] = None,
	link_color : Union[str, list[float]] = None,
	link_greyout_opacity : float = None,
	link_width : float = None,
	link_width_scale : float = None,
	link_arrows : bool = None,
	link_arrows_size_scale : float = None,
	link_visibility_distance_range : list[float] = None,
	link_visibility_min_transparency : float = None,
	link_source_by : str = None,
	link_sou

In [82]:
print(f"{old_sig == new_sig = }")

old_sig == new_sig = True


# Partial cosmo

In [2]:
from cosmograph import cosmo
import pandas as pd

In [9]:
src = '/Users/thorwhalen/Dropbox/_odata/figiri/wordnet_words/wordnet_feature_meta.parquet'

df = pd.read_parquet(src)
print(f"{df.shape=}")
df.iloc[0]

df.shape=(123587, 8)


word                                                          a
frequency                                              0.015441
definition    a metric unit of length equal to one ten billi...
lexname                                           noun.quantity
name                                              angstrom.n.01
pos                                                        noun
umap_x                                                 3.027916
umap_y                                                 3.760965
Name: angstrom.n.01.a, dtype: object

In [5]:
data = {
    'Quest_Title': [
        'Dragon Hunt',
        'Mystic Voyage',
        'Treasure Seekers'
    ],
    'Map_X': [120, 450, 100],
    'Map_Y': [75, 320, 210],
    'Aura_Color': ['Crimson', 'Azure', 'Emerald'],
    'Gear_Size': [15, 20, 10]
}

# Create the DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)

cosmo(
    df,
    point_x_by='Map_X',
    point_y_by='Map_Y',
    point_color_by='Aura_Color',
    point_size_by='Gear_Size',
    point_size_scale=7,
    point_label_by='Quest_Title',
)

        Quest_Title  Map_X  Map_Y Aura_Color  Gear_Size
0       Dragon Hunt    120     75    Crimson         15
1     Mystic Voyage    450    320      Azure         20
2  Treasure Seekers    100    210    Emerald         10


Cosmograph(background_color=None, focused_point_ring_color=None, hovered_point_ring_color=None, link_color=Non…

In [4]:
# Example 2: Fantasy Potion Brewing

# Define the data with whimsical column names
data_potions = {
    'Potion_Name': [
        'Elixir of Vitality',
        'Draught of Invisibility',
        'Brew of Fireball'
    ],
    'Brewmaster': [
        'Elder Willow',
        'Mistress Shadow',
        'Master Ignis'
    ],
    'Ingredients': [
        'Dragon Scale, Phoenix Feather, Unicorn Tears',
        'Nightshade, Moonflower, Shadow Essence',
        'Fireroot, Lava Crystal, Ember Dust'
    ],
    'Location_X': [500, 620, 450],
    'Location_Y': [300, 400, 350],
    'Potion_Color': ['Golden', 'Transparent', 'Crimson'],
    'Potion_Size': [25, 30, 20]
}

# Create the DataFrame
df_potions = pd.DataFrame(data_potions)

# Display the DataFrame
print(df_potions)





               Potion_Name       Brewmaster  \
0       Elixir of Vitality     Elder Willow   
1  Draught of Invisibility  Mistress Shadow   
2         Brew of Fireball     Master Ignis   

                                    Ingredients  Location_X  Location_Y  \
0  Dragon Scale, Phoenix Feather, Unicorn Tears         500         300   
1        Nightshade, Moonflower, Shadow Essence         620         400   
2            Fireroot, Lava Crystal, Ember Dust         450         350   

  Potion_Color  Potion_Size  
0       Golden           25  
1  Transparent           30  
2      Crimson           20  


In [4]:
import pandas as pd
from cosmograph import cosmo

single_link = {
    'points': [{'id': '0'}, {'id': '1'}],
    'links': [{'source': '0', 'target': '1'}],
}

points, links = map(pd.DataFrame, single_link.values())

g = cosmo(
    points=[{'id': '0'}, {'id': '1'}],
    links=[{'source': '0', 'target': '1'}],
    point_id_by='id',
    link_source_by='source',
    link_target_by='target',
    simulation_gravity=0,
    simulation_center=1,
)

AttributeError: 'list' object has no attribute 'select_dtypes'